
# CT421 Project 1  Evolutionary Search - GAs

**Aoife Mulligan 20307646 | Leo Chui 20343266**  

[Useful Webpage](https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/)

### 1.1 OneMax Problem

In [ ]:
import random

# Define global variables for string size, population size, and mutation rate

STRING_SIZE = 30
POPULATION_SIZE = 100
MUTATION_RATE = 1/STRING_SIZE


In [3]:
# Generate random binary string of length 30

def generate_binary_string():
    return ''.join(random.choice('01') for _ in range(30))

population = [generate_binary_string() for _ in range(10)]
print(population)

'''for i in range(10):
    print(generate_binary_string())
    i += 1'''



['110101010101001001000100101100', '101110111111001001111111011100', '101010000101010111110111100011', '101001010111000011010101010000', '100011101101001101011100101000', '100011101111111010010010000111', '000111111100001111001100001111', '011011111100001011100001001001', '000011100101010100100110100100', '001101000000011010101101001010']


'for i in range(10):\n    print(generate_binary_string())\n    i += 1'

In [ ]:
# Create a function which returns 999 if the list has all 1's, 
# and otherwise returns the number of 1's in the string.

def fitness(ans):
    
    # count the number of 1's in the string
    count = ans.count('1')

    if count == 30:
        return 999
    else:
        return count



In [ ]:
'''scores = [fitness(entity) for entity in population]
print(scores)'''

# Evaluate population fitness and return the fittest 5 individuals

def evaluate_population(population):
    scores = [fitness(entity) for entity in population]
    # Create pairs of scores and population, and sort by score
    sorted_population = [x for _, x in sorted(zip(scores, population), reverse=True)]
    # Returns the fittest 5
    return sorted_population[:5]
    

In [ ]:
# Merge the first 5 individuals in the population to create 10 children

def merge(population):
    children = []
    for i in range(5):
        # Merge the first half of the first individual with the second half of the second individual
        children.append(population[i] + population[i+1])
    return children

In [ ]:
# Randomly mutate each bit in each child with probability MUTATION_RATE

def mutate(children):
    mutated_children = []
    for child in children:
        mutated_child = ''
        for bit in child:
            if random.random() < MUTATION_RATE:
                mutated_child += str(1 - int(bit))
            else:
                mutated_child += bit
        mutated_children.append(mutated_child)
    return mutated_children